In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb




In [ ]:
assets=['PETR4.SA','ABEV3.SA','CIEL3.SA','ITUB4.SA','^BVSP']
portfolio=pd.DataFrame()

In [88]:
import datetime
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 6, 15)

In [89]:
for t in assets:   
    portfolio[t]=wb.DataReader(t,data_source='yahoo',start='2012-1-1',end='2019-6-20')['Adj Close']

sec_ret=np.log(portfolio/portfolio.shift(1))


In [90]:
sec_ret.tail()

,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA,^BVSP
Date,,,,,
2019-06-14,-0.004425,-0.011835,-0.009050,0.000294,-0.007459
2019-06-17,0.001846,0.009029,0.003026,-0.001178,-0.004262
2019-06-18,0.012464,0.013947,0.019447,0.025590,0.018079
2019-06-19,0.002547,0.008276,0.014706,0.015668,0.009003
2019-06-21,0.027242,0.013643,0.044261,0.017372,0.016905


## Matriz de covariância

In [91]:
# Vamos assumir 250 dias ao ano negociados.
covariance=sec_ret.cov()*250
covariance


,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA,^BVSP
PETR4.SA,0.233129,0.032836,0.041939,0.084750,0.083534
ABEV3.SA,0.032836,0.052942,0.022287,0.027608,0.025215
CIEL3.SA,0.041939,0.022287,0.105912,0.032540,0.029828
ITUB4.SA,0.084750,0.027608,0.032540,0.091760,0.055721
^BVSP,0.083534,0.025215,0.029828,0.055721,0.050604


## Índice Beta: Como é Calculado?

A fórmula do Índice Beta é bem simples:

Beta = Covariância entre o Retorno do Ativo e do Mercado / Variância do Retorno do Mercado

In [92]:
marketcov = covariance.iloc[:,-1]
market_var=sec_ret.var()*250

In [94]:
marketcov

PETR4.SA    0.083534
ABEV3.SA    0.025215
CIEL3.SA    0.029828
ITUB4.SA    0.055721
^BVSP       0.050604
Name: ^BVSP, dtype: float64

In [93]:
betam = marketcov/market_var
betam

PETR4.SA    0.358314
ABEV3.SA    0.476267
CIEL3.SA    0.281629
ITUB4.SA    0.607245
^BVSP       1.000000
dtype: float64

In [96]:
# Expected Return
premio = 0.065 - 0.02
Exp_ret= 0.065+betam*premio
Exp_ret

PETR4.SA    0.081124
ABEV3.SA    0.086432
CIEL3.SA    0.077673
ITUB4.SA    0.092326
^BVSP       0.110000
dtype: float64

## Índice de Sharpe: Como é Calculado?

IS = ( Ri -Rf ) / (σi )

Onde:

    IS = Índice de Sharpe
    Ri = Retorno do Ativo
    Rf = Retorno Livre de Riso ( Risk-Free )
    σi = Risco do Ativo ( a letra grega sigma representa volatilidade )


In [58]:
# Calculation os Sharpe ratio

Rf = 0.065   #SELIC

sharpes =[]
for t in assets:
    
    sigma = (sec_ret['ITUB4.SA'].std()*250**0.5)

    sharpe=(Exp_ret-Rf)/sigma
sharpe

-0.028884308990665287